In [1]:
# import pickle

# # Open the .pkl file in binary read mode
# with open('/kaggle/input/ucf-crime-video-dataset/UCF-Crime/frames.pkl', 'rb') as file:
#     # Load the content from the .pkl file
#     loaded_data = pickle.load(file)

In [13]:
# len(loaded_data)

1949

In [2]:
# import numpy as np

# # Load the .npy file
# loaded_array = np.load('/kaggle/input/ucf-crime-video-dataset/UCF-Crime/all_rgbs/Arrest/Arrest002_x264.mp4.npy')

# # Print the shape of the array
# print("Shape of the array:", loaded_array.shape)

Shape of the array: (32, 1024)


In [1]:
import os
# import pickle
import numpy as np
import random
import torch
import torch.nn as nn

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
def read_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
from tqdm.auto import tqdm
from sklearn import metrics
from torch.nn.parallel import DataParallel
import matplotlib.pyplot as plt

set_random_seed(21)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import wandb
wandb.login(key = '02c8923278a3dc82932fafb9959cd6d7587dacc7')
# 02c8923278a3dc82932fafb9959cd6d7587dacc7

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# set configuration, hyperparameters
config = dict(
    epochs=70,
    classes=2,
    learning_rate=0.001,
    dataset="UCF-Crime",
    architecture="MIL")

best_auc = 0 

In [3]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import random
import multiprocessing

class Normal_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """
    def __init__(self, is_train=1, path='/kaggle/input/ucf-crime-video-dataset/UCF-Crime/'):
        super(Normal_Loader, self).__init__()
        self.is_train = is_train
        self.path = path
        if self.is_train == True:
            data_list = os.path.join(path, 'train_normal.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_normalv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
            random.shuffle(self.data_list)
            self.data_list = self.data_list[:-10]
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == True:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split(' ')[0], int(self.data_list[idx].split(' ')[1]), int(self.data_list[idx].split(' ')[2][:-1])
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy, gts, frames

class Anomaly_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """
    def __init__(self, is_train=True, path='/kaggle/input/ucf-crime-video-dataset/UCF-Crime/'):
        super(Anomaly_Loader, self).__init__()
        self.is_train = is_train
        self.path = path
        if self.is_train == True:
            data_list = os.path.join(path, 'train_anomaly.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_anomalyv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == True:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split('|')[0], int(self.data_list[idx].split('|')[1]), self.data_list[idx].split('|')[2][1:-2].split(',')
            gts = [int(i) for i in gts]
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy, gts, frames


def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True,
                                         num_workers=multiprocessing.Pool()._processes)
    return loader

In [6]:
import torch.nn.functional as F

def MIL(y_pred, batch_size, is_transformer=0):
    loss = torch.tensor(0.).cuda()
    loss_intra = torch.tensor(0.).cuda()
    sparsity = torch.tensor(0.).cuda()
    smooth = torch.tensor(0.).cuda()
    if is_transformer==0:
        y_pred = y_pred.view(batch_size, -1)
    else:
        y_pred = torch.sigmoid(y_pred)

    for i in range(batch_size):
        anomaly_index = torch.randperm(30).cuda()
        normal_index = torch.randperm(30).cuda()

        y_anomaly = y_pred[i, :32][anomaly_index]
        y_normal  = y_pred[i, 32:][normal_index]

        y_anomaly_max = torch.max(y_anomaly) # anomaly
        y_anomaly_min = torch.min(y_anomaly)

        y_normal_max = torch.max(y_normal) # normal
        y_normal_min = torch.min(y_normal)

        loss += F.relu(1.-y_anomaly_max+y_normal_max)

        sparsity += torch.sum(y_anomaly)*0.00008
        smooth += torch.sum((y_pred[i,:31] - y_pred[i,1:32])**2)*0.00008
    loss = (loss+sparsity+smooth)/batch_size

    return loss

In [7]:
class Learner(nn.Module):
    def __init__(self, input_dim=2048, drop_p=0.0):
        super(Learner, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.drop_p = 0.6
        self.weight_init()
        self.vars = nn.ParameterList()

        for i, param in enumerate(self.classifier.parameters()):
            self.vars.append(param)

    def weight_init(self):
        for layer in self.classifier:
            if type(layer) == nn.Linear:
                nn.init.xavier_normal_(layer.weight)

    def forward(self, x, vars=None):
        if vars is None:
            vars = self.vars
        x = F.linear(x, vars[0], vars[1])
        x = F.relu(x)
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[2], vars[3])
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[4], vars[5])
        return torch.sigmoid(x)

    def parameters(self):
        """
        override this function since initial parameters will return with a generator.
        :return:
        """
        return self.vars

In [8]:
def make():
    # Make the data
    normal_train_dataset = Normal_Loader(is_train=1)
    normal_test_dataset = Normal_Loader(is_train=0)

    anomaly_train_dataset = Anomaly_Loader(is_train=1)
    anomaly_test_dataset = Anomaly_Loader(is_train=0)

    normal_train_loader = make_loader(normal_train_dataset, batch_size=30)
    normal_test_loader = make_loader(normal_test_dataset, batch_size=1)

    anomaly_train_loader = make_loader(anomaly_train_dataset, batch_size=30) 
    anomaly_test_loader = make_loader(anomaly_test_dataset, batch_size=1)

    # Make the model
    model = Learner(input_dim=2048, drop_p=0.0).to(device)
    # model = DataParallel(model)
    optimizer = torch.optim.Adagrad(model.parameters(), lr= 0.001, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50])
    criterion = MIL
    # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-6, max_lr=0.1, mode='triangular', gamma=1.0, cycle_momentum=False)
    # cycle_momentum must be set to False if we want to work CyclicLR with Adam optimizer
    
    return model, normal_train_loader, normal_test_loader, anomaly_train_loader, anomaly_test_loader, criterion, scheduler, optimizer

In [53]:
def train(model, normal_train_loader, normal_test_loader, anomaly_train_loader, anomaly_test_loader, criterion, optimizer, scheduler, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(normal_train_loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    batch_loss = 0
    for epoch in tqdm(range(config.epochs)):
        model.train()
        train_loss = 0
        for batch_idx, (normal_inputs, anomaly_inputs) in enumerate(zip(normal_train_loader, anomaly_train_loader)):
            inputs = torch.cat([anomaly_inputs, normal_inputs], dim=1)
            batch_size = inputs.shape[0]
            inputs = inputs.view(-1, inputs.size(-1)).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, batch_size)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            example_ct += batch_size
            batch_loss = train_loss/len(normal_train_loader)
        train_log(batch_loss, scheduler, example_ct, epoch)
        scheduler.step()
        test(model, normal_test_loader, anomaly_test_loader)
    
def train_log(loss, lr_scheduler, example_ct, epoch):
    wandb.log({"epoch": epoch,'lr': lr_scheduler.get_last_lr()[0], "loss": loss}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [54]:
def test(model, normal_test_loader, anomaly_test_loader):
    model.eval()
    auc = 0
    global best_auc
    all_gt_list = []
    all_score_list = []
    with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)):
            inputs, gts, frames = data
            inputs = inputs.view(-1, inputs.size(-1)).to(torch.device('cuda'))
            score = model(inputs)
            score = score.cpu().detach().numpy()
            score_list = np.zeros(frames[0])
            step = np.round(np.linspace(0, frames[0]//16, 33))

            for j in range(32):
                score_list[int(step[j])*16:(int(step[j+1]))*16] = score[j]

            gt_list = np.zeros(frames[0])
            for k in range(len(gts)//2):
                s = gts[k*2]
                e = min(gts[k*2+1], frames)
                gt_list[s-1:e] = 1

            inputs2, gts2, frames2 = data2
            inputs2 = inputs2.view(-1, inputs2.size(-1)).to(torch.device('cuda'))
            score2 = model(inputs2)
            score2 = score2.cpu().detach().numpy()
            score_list2 = np.zeros(frames2[0])
            step2 = np.round(np.linspace(0, frames2[0]//16, 33))
            for kk in range(32):
                score_list2[int(step2[kk])*16:(int(step2[kk+1]))*16] = score2[kk]
            gt_list2 = np.zeros(frames2[0])
            score_list3 = np.concatenate((score_list, score_list2), axis=0)
            gt_list3 = np.concatenate((gt_list, gt_list2), axis=0)

            fpr, tpr, thresholds = metrics.roc_curve(gt_list3, score_list3, pos_label=1)
            auc += metrics.auc(fpr, tpr)
            all_gt_list.extend(gt_list3)
            all_score_list.extend(score_list3)
        final_fpr, final_tpr, final_thresholds = metrics.roc_curve(all_gt_list, all_score_list, pos_label=1)
        # print('auc = ', auc/140)

        if best_auc < auc/140:
            print('auc = {}',auc/140) 
            print('Saving..')
            state = {
                'net': model.state_dict(),
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt.pth')
            best_auc = auc/140
            
        wandb.log({"auc_accuracy": auc / 140})
    return final_fpr, final_tpr

    # Save the model in the exchangeable ONNX format
    # torch.onnx.export(model, images, "model.onnx")
    # wandb.save("model.onnx")

In [55]:
def plot_roc(fpr, tpr):
    plt.figure()
    lw = 2 # linewidth
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, color='blue', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='pink', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.savefig("roc_curve.png")

    print('ROC curve (area = %0.2f)' % roc_auc)

In [56]:
def model_pipeline(hyperparameters):
    # tell wandb to get started
    with wandb.init(project="UCF32", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config
        
      # make the model, data, and optimization problem
        model, normal_train_loader, normal_test_loader, anomaly_train_loader, anomaly_test_loader, criterion, scheduler, optimizer = make()
        print(model)

      # and use them to train the model
        train(model, normal_train_loader, normal_test_loader, anomaly_train_loader, anomaly_test_loader, criterion, optimizer, scheduler, config)

      # and test its final performance
        fpr, tpr = test(model, normal_test_loader, anomaly_test_loader)
        plot_roc(fpr, tpr)
        wandb.log({"ROC Curve": wandb.Image("roc_curve.png")})
    return model

In [19]:
# wandb.init(project="UCF32")

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model_pipeline(config)

DataParallel(
  (module): Learner(
    (classifier): Sequential(
      (0): Linear(in_features=2048, out_features=512, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.6, inplace=False)
      (3): Linear(in_features=512, out_features=32, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.6, inplace=False)
      (6): Linear(in_features=32, out_features=1, bias=True)
      (7): Sigmoid()
    )
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 512x2048 (GPU 0)]
        (1): Parameter containing: [torch.float32 of size 512 (GPU 0)]
        (2): Parameter containing: [torch.float32 of size 32x512 (GPU 0)]
        (3): Parameter containing: [torch.float32 of size 32 (GPU 0)]
        (4): Parameter containing: [torch.float32 of size 1x32 (GPU 0)]
        (5): Parameter containing: [torch.float32 of size 1 (GPU 0)]
    )
  )
)


  0%|          | 0/70 [00:00<?, ?it/s]

Loss after 00810 examples: 0.978
Loss after 01620 examples: 0.935
Loss after 02430 examples: 0.889
Loss after 03240 examples: 0.852
Loss after 04050 examples: 0.817
Loss after 04860 examples: 0.787
Loss after 05670 examples: 0.764
Loss after 06480 examples: 0.727
Loss after 07290 examples: 0.710
Loss after 08100 examples: 0.687
Loss after 08910 examples: 0.660
Loss after 09720 examples: 0.648
Loss after 10530 examples: 0.632
Loss after 11340 examples: 0.604
Loss after 12150 examples: 0.588
Loss after 12960 examples: 0.575
Loss after 13770 examples: 0.563
Loss after 14580 examples: 0.547
Loss after 15390 examples: 0.527
Loss after 16200 examples: 0.525
Loss after 17010 examples: 0.511
Loss after 17820 examples: 0.506
Loss after 18630 examples: 0.498
Loss after 19440 examples: 0.481
Loss after 20250 examples: 0.476
Loss after 21060 examples: 0.470
Loss after 21870 examples: 0.478
Loss after 22680 examples: 0.469
Loss after 23490 examples: 0.471
Loss after 24300 examples: 0.463
Loss after

In [43]:
# file_path = '/kaggle/input/ucf-crime-video-dataset/UCF-Crime/test_normalv2.txt'

# # Counting the number of lines in the file
# with open(file_path, 'r') as file:
#     line_count = sum(1 for line in file)

# print(f"Number of lines in the file: {line_count}")

Number of lines in the file: 150
